### Pure cutting planes approach for TSP!

In [ ]:
!pip install mip

In [1]:
from itertools import product
from networkx import minimum_cut, DiGraph
from mip import Model, xsum, BINARY, OptimizationStatus, CutType

In [2]:
N = ["a", "b", "c", "d", "e", "f", "g"]
A = { ("a", "d"): 56, ("d", "a"): 67, ("a", "b"): 49, ("b", "a"): 50,
      ("f", "c"): 35, ("g", "b"): 35, ("g", "b"): 35, ("b", "g"): 25,
      ("a", "c"): 80, ("c", "a"): 99, ("e", "f"): 20, ("f", "e"): 20,
      ("g", "e"): 38, ("e", "g"): 49, ("g", "f"): 37, ("f", "g"): 32,
      ("b", "e"): 21, ("e", "b"): 30, ("a", "g"): 47, ("g", "a"): 68,
      ("d", "c"): 37, ("c", "d"): 52, ("d", "e"): 15, ("e", "d"): 20,
      ("d", "b"): 39, ("b", "d"): 37, ("c", "f"): 35}

In [3]:
Aout = {n : [a for a in A if a[0]==n] for n in N}
Ain  = {n : [a for a in A if a[1]==n] for n in N}

In [6]:
m = Model()

x = {a : m.add_var(name = "x({},{})".format(a[0],a[1]), var_type = BINARY) for a in A}

m.objective = xsum(c*x[a] for a,c in A.items())

for n in N:
    m += xsum(x[a] for a in Aout[n]) == 1, "out({})".format(n)
    m += xsum(x[a] for a in Ain[n])  == 1, "in({})".format(n)
    
newConstraints = True

while newConstraints:
    m.optimize(relax = True)
    print("status: {}, objective value: {}".format(m.status, m.objective_value))
    
    G = DiGraph()
    for a in A:
        G.add_edge(a[0], a[1], capacity = x[a].x)
        
    newConstraints = False
    for (n1,n2) in [(i, j) for (i, j) in product(N, N) if i!= j]:
        cut_value, (S, NS) = minimum_cut(G, n1, n2)
        if cut_value <= 0.99:
            m += (xsum(x[a] for a in A if (a[0] in S and a[1] in S)) <= len(S) - 1)
            newConstraints = True
    if not newConstraints and m.solver_name.lower() == 'cbc':
        cp = m.generate_cuts([CutType.GOMORY, CutType.MIR, CutType.ZERO_HALF, CutType.KNAPSACK_COVER])
        
        if cp.cuts:
            m += cp
            newConstraints = True
        

status: OptimizationStatus.OPTIMAL, objective value: 237.0
status: OptimizationStatus.OPTIMAL, objective value: 261.0
status: OptimizationStatus.OPTIMAL, objective value: 261.0
status: OptimizationStatus.OPTIMAL, objective value: 262.0


### Branch and Cut TSP!

In [2]:
from typing import List, Tuple
from random import seed, randint
from itertools import product
from math import sqrt
import networkx as nx
from mip import Model, xsum, BINARY, minimize, ConstrsGenerator, CutPool

In [13]:
class SubTourCutGenerator(ConstrsGenerator):
    def __init__(self, Fl: List[Tuple[int, int]], x_, V_):
        self.F, self.x, self.V = Fl, x_, V_
        
    def generate_constrs(self, model: Model, depth: int = 0, npass: int = 0):
        xf, V_, cp, G = model.translate(self.x), self.V, CutPool(), nx.DiGraph()
        
        for (u,v) in [(k,l) for (k,l) in product(V_, V_) if k!=l and xf[k][l]]:
            G.add_edge(u, v, capacity = xf[u][v].x)
        
        for (u,v) in F:
            val, (S, NS) = nx.minimum_cut(G, u, v)
            if val <= 0.99:
                aInS = [(xf[i][j], xf[i][j].x) for (i,j) in product (V_, V_) if i != j and xf[i][j] and i in S and j in S]
                if sum(f for v, f in aInS) >= (len(S) - 1)+1e-4:
                    cut = xsum(1.0*v for v, fm in aInS) <= len(S) - 1
                    cp.add(cut)
                    if len(cp.cuts) > 256:
                        for cut in cp.cuts:
                            model += cut
                            
                        return
        
        for cut in cp.cuts:
            model += cut
            
                        
        

In [14]:
n = 30 # number of points
V = set(range(n))
seed(0)
p = [(randint(1, 100), randint(1,100)) for i in V] # coordinates
Arcs = [(i,j) for (i,j) in product(V, V) if i != j]

# distance matrix
c = [[round(sqrt((p[i][0]-p[j][0])**2 + (p[i][1]-p[j][1])**2)) for j in V] for i in V]

model = Model()

# binary variable indicating arc(i,j) is used on the route or not
x = [[model.add_var(var_type = BINARY) for j in V] for i in V]

# continuous variable to prevent the sub tour and each city will have a different sequential id in the planned route
y = [model.add_var() for i in V]

# objective function minimizes the distance

model.objective = minimize(xsum(c[i][j]*x[i][j] for (i,j) in Arcs))

# constraints : leave each city only once
for i in V:
    model += xsum(x[i][j] for j in V - {i}) == 1

# constraints : enter each city once
for i in V:
    model += xsum(x[j][i] for j in V - {i}) == 1
    
# subtour elimination

for (i,j) in product(V - {0}, V - {0}):
    if i != j:
        model += y[i] - (n+1)*x[i][j] >= y[j] - n
        
# no subtours of size 2

for (i,j) in Arcs:
    model += x[i][j] + x[j][i] <= 1
    
# computing farthest point to each node and this will be checked first for isolated subtours

F, G = [], nx.DiGraph()

for (i,j) in Arcs:
    G.add_edge(i, j, weight = c[i][j])

for i in V:
    P, D = nx.dijkstra_predecessor_and_distance(G, source = i)
    DS = list(D.items())
    DS.sort(key = lambda x:x[1])
    F.append((i, DS[-1][0]))
    

model.cuts_generator = SubTourCutGenerator(F, x, V)
model.optimize()

print('status : %s route length %g ' % (model.status, model.objective_value ))

status : OptimizationStatus.OPTIMAL route length 400 
